In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = '/Users/patrickokwir/Desktop/Git_Projects/National-Collision-Database-Analysis/dataset/y_2017_en.xlsx'
df = pd.read_excel(data)
df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
0,2017,1,1,10,2,1,2,3,1,3,...,1,UUUU,1,M,75,11,2,NN,1,2456013
1,2017,1,1,12,2,1,4,UU,1,5,...,1,UUUU,1,F,21,11,2,2,1,2456022
2,2017,1,1,0,2,1,3,UU,7,3,...,1,UUUU,1,F,34,11,2,2,1,2456061
3,2017,1,1,17,2,2,21,UU,1,1,...,1,UUUU,1,F,50,11,2,2,1,2456167
4,2017,1,1,17,2,2,21,UU,1,1,...,1,UUUU,1,M,63,11,1,NN,1,2456167


**Question 2: Using Excel (Pivot Tables/Charts), or any other analytical tool of your choice, generate reports/charts to answer the following questions:**

What age range and which sex are more likely to be associated with a collision?\
What time(s) of days are most associated with a relatively high fatality rate?\
What type(s) of weather are most associated with a relatively high fatality rate?\
What is the effect of using a Safety device on the fatality rate?\
Use the previous charts/reports and perform additional ad hoc analysis of the dataset to outline the key contributing factors to Canadian collisions.

**2.1 What age range and which sex are more likely to be associated with a collision?\**

In [3]:
df['P_SEX'].unique()

array(['M', 'F', 'U', 'N'], dtype=object)

In [4]:
df['P_AGE'].unique()

array([75, 21, 34, 50, 63, 26, 20, 80, 35, 37, 45, 32, 38, 60, 25, 19, 27,
       43, 72, 55, 31, 76, 'UU', 51, 46, 42, 58, 59, 56, 41, 36, 48, 39,
       22, 52, 53, 33, 18, 61, 40, 29, 3, 73, 69, 30, 44, 24, 54, 65, 47,
       70, 71, 13, 23, 17, 74, 15, 66, 88, 57, 49, 67, 14, 64, 28, 68, 77,
       62, 1, 8, 2, 11, 9, 16, 89, 85, 84, 4, 6, 81, 12, 5, 7, 78, 79, 93,
       86, 10, 91, 90, 83, 92, 82, 87, 'NN', 99, 98, 97, 96, 94, 95],
      dtype=object)

In [5]:
df['C_SEV'].unique()

array([2, 1])

In [12]:
# Filter out rows with 'Unknown' values in 'P_SEX' and convert non-numeric 'P_AGE' values to NaN
df = df[df['P_SEX'] != 'U']
df = df[df['P_SEX']!= 'N']
df = df[df['P_AGE']!= 'UU']
df = df[df['P_AGE']!= 'NN']
df['P_AGE'] = pd.to_numeric(df['P_AGE'], errors='coerce')

# Filter rows where collision severity is either 1 or 2
collision_df = df[df['C_SEV'].isin([1, 2])]

# Define age ranges and corresponding labels
age_ranges = [(0, 17), (18, 25), (26, 33), (34, 41), (41, 47), (48, 51), (52, 59), (60, 100)]
age_range_labels = ['0-17', '18-25', '26-33', '34-41', '47 -48', '49-51', '52-59', '60+']

# Define Sex Labels for each P_SEX value and replace in df
collision_df['P_SEX'] = collision_df['P_SEX'].replace({'M': 'Male', 'F': 'Female'})


# Define Severity Labels for each C_SEV value and replace in collision_df
collision_df['C_SEV'] = collision_df['C_SEV'].replace({'1': 'At least one fatality', '2': 'Non-Fatal'})

# Calculate total collisions for each age range and sex combination
collision_counts = collision_df.groupby(['P_SEX', pd.cut(collision_df['P_AGE'], [age[0] for age in age_ranges] + [age_ranges[-1][1]])]).size().unstack().fillna(0)
collision_proportions = collision_counts.div(collision_counts.sum(axis=1), axis=0)

# Find the age range and sex with the highest collision proportion
max_collision_proportion = collision_proportions.stack().max()
max_collision_idx = collision_proportions.stack().idxmax()

print(f"The age range and sex more likely to be associated with a collision:")
print(f"Age Range: {max_collision_idx[1]}, Sex: {max_collision_idx[0]}")
print(f"Collision Proportion: {max_collision_proportion:.2%}")



The age range and sex more likely to be associated with a collision:
Age Range: (18, 26], Sex: Female
Collision Proportion: 18.28%


In [7]:
from lets_plot import *
LetsPlot.setup_html()

In [25]:
# get data to plot 
plot_data = collision_proportions.transpose().reset_index(level=0)
plot_data

P_SEX,P_AGE,Female,Male
0,"(0, 18]",0.141919,0.125006
1,"(18, 26]",0.182830,0.181522
2,"(26, 34]",0.144212,0.147055
3,"(34, 41]",0.113182,0.109846
4,"(41, 48]",0.104819,0.105264
5,"(48, 52]",0.057622,0.061145
6,"(52, 60]",0.105289,0.116265
7,"(60, 100]",0.150126,0.153898


In [27]:
# plot the data using ggplot2

2.2 What time(s) of days are most associated with a relatively high fatality rate?

In [ ]:
# Convert 'C_HOUR' and 'P_ISEV' columns to numeric
df['C_HOUR'] = pd.to_numeric(df['C_HOUR'], errors='coerce')
df['P_ISEV'] = pd.to_numeric(df['P_ISEV'], errors='coerce')

# Filter out non-numeric values in 'C_HOUR' and 'P_ISEV'
df = df.dropna(subset=['C_HOUR', 'P_ISEV'])

# Filter rows where 'P_ISEV' indicates fatality (3)
fatality_df = df[df['P_ISEV'] == 3]

# Group by 'C_HOUR' and calculate the fatality rate for each hour
fatality_rate_by_hour = fatality_df.groupby('C_HOUR').size() / df.groupby('C_HOUR').size()

# Plot the fatality rate by hour
plt.bar(fatality_rate_by_hour.index, fatality_rate_by_hour.values)
plt.xlabel('Time of Day (Hour)')
plt.ylabel('Fatality Rate')
plt.title('Fatality Rate by Time of Day')
plt.xticks(fatality_rate_by_hour.index)

# Find the hour with the highest fatality rate
max_fatality_rate_hour = fatality_rate_by_hour.idxmax()
max_fatality_rate = fatality_rate_by_hour.max()

print(f"Time of Day Most Associated with High Fatality Rate:")
print(f"Hour: {max_fatality_rate_hour}, Fatality Rate: {max_fatality_rate:.2%}")

plt.show()